# 环境

In [1]:
import torch
import torchmetrics
import torchvision
import os
import gc
import torch.nn as nn
from torchvision import transforms
print(torch.__version__)
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
if device=='cuda':
    gc.collect()
    torch.cuda.empty_cache()
print(os.getcwd())


1.13.1
cuda
d:\project\deep-learning\unsupervised denoise


# 模型

In [25]:
class HourGlassCNNBlock(nn.Module):
    def __init__(self,in_channels,out_channels,size=(128,128),layernorm=False,batchnorm=False,activation=True):
        super(HourGlassCNNBlock,self).__init__()
        layers=[]
        layers.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1))
        if layernorm:layers.append(nn.LayerNorm([out_channels,size[0],size[1]]))
        if batchnorm:layers.append(nn.BatchNorm2d(out_channels))
        if activation:layers.append(nn.SiLU())
        self.main=nn.Sequential(*layers)
    def forward(self,x):
        return self.main(x)#channel不同不能进行残差学习

class HourGlassCNN(nn.Module):
    def __init__(self,size=(128,128),layernorm=False,batchnorm=False):
        super(HourGlassCNN,self).__init__()
        self.blocks=nn.ModuleList([
            HourGlassCNNBlock(3,64,size,layernorm=False,batchnorm=False),#0
            HourGlassCNNBlock(64,128,size,layernorm,batchnorm),#1
            HourGlassCNNBlock(128,256,size,layernorm,batchnorm),#2
            HourGlassCNNBlock(256,512,size,layernorm,batchnorm),#3
            HourGlassCNNBlock(512,1024,size,layernorm,batchnorm),#4

            HourGlassCNNBlock(1024,512,size,layernorm,batchnorm),#5
            HourGlassCNNBlock(512,256,size,layernorm,batchnorm),#6
            HourGlassCNNBlock(256,128,size,layernorm,batchnorm),#7
            HourGlassCNNBlock(128,64,size,layernorm,batchnorm),#8
            HourGlassCNNBlock(64,3,size,layernorm=False,batchnorm=False,activation=False),#9
        ])
    def forward(self,x):
        y0=self.blocks[0](x)
        y1=self.blocks[1](y0)
        y2=self.blocks[2](y1)
        y3=self.blocks[3](y2)

        y4=self.blocks[4](y3)

        y5=self.blocks[5](y4)+y3#残差
        y6=self.blocks[6](y5)+y2
        y7=self.blocks[7](y6)+y1
        y8=self.blocks[8](y7)+y0
    
        y9=self.blocks[9](y8)+x
        return y9
    def loss(self,input,target):
        return nn.functional.smooth_l1_loss(self(input),target,beta=0.1)

# 分析

In [27]:
import torch.utils.tensorboard
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.notebook import tqdm
from torchvision.utils import make_grid
from matplotlib import pyplot as plt
from torchmetrics import PeakSignalNoiseRatio

In [28]:
def convert_to_rgb255(image:torch.Tensor):
    image = (image + 1) / 2
    image[image < 0] = 0
    image[image > 1] = 1
    return image
def show_image(image:torch.Tensor):
    image=convert_to_rgb255(image)
    plt.imshow(transforms.ToPILImage()(image))
    plt.show()

writer=SummaryWriter()
psnr=PeakSignalNoiseRatio().to(device=device)

# 数据

In [29]:
img_size=128
batch_size=8

In [30]:

from torch.utils.data import Dataset 
from torch.utils.data import DataLoader

from PIL import Image
# https://discuss.pytorch.org/t/torchvision-transfors-how-to-perform-identical-transform-on-both-image-and-target/10606/7

class BSDSPairsDataSet(Dataset):
    def __init__(self,dir=None,img_size=(128,128),sigma=30) -> None:
        super(BSDSPairsDataSet,self).__init__()
        if not dir:dir='./data/BSDS300/%dx%d/base'%img_size
        self.img_size=img_size
        self.paths=[dir+'/'+file for file in os.listdir(dir)]
        self.sigma=sigma
    def __getitem__(self,idx:int):
        img_path = os.path.join(self.images_dir, self.files[idx])
        clean = Image.open(img_path).convert('RGB')#比torchvision的好
        transform = transforms.Compose([
            # hwc->chw 并 归一化到[0,1]
            transforms.ToTensor(),
            # [−1,1]
            transforms.Normalize((.5, .5, .5), (.5, .5, .5)),
            ])
        clean = transform(clean)
        noisy = clean + self.sigma * torch.randn(clean.shape)/255
        return noisy, clean
    def __len__(self):
        return len(self.paths)

train_set=BSDSPairsDataSet()
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=False)


# 训练

In [ ]:
import itertools
from torch.optim import Adam
noise_extractor = HourGlassCNN(layernorm=True,batchnorm=True).to(device)
restomer = HourGlassCNN(layernorm=True,batchnorm=True).to(device)
optimizer = Adam(itertools.chain(noise_extractor.parameters(),restomer.parameters()) , lr=1e-3)

In [ ]:
# noise as clean：noisy-P'->noisy' 如果clean-P->noisy中的P近似于P'那么(noisy',noisy)训练可以近似复原噪声图片
def train(input):
    noises=noise_extractor(input)-input
    total_loss=None
    for i in range(batch_size):
        for noisys,_ in train_loader:
            noisys=noisys.to(device=device)
            optimizer.zero_grad()
            loss=restomer.loss(noisys+noises[i],noisys)
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                total_loss+=loss.sum()#返回tensor，要避免计算图不释放，导致内存溢出
    return total_loss
def draw(mean_loss_noisy,mean_loss_clean,noisys,cleans,epoch):
    outs=restomer(noisys)
    writer.add_scalar('noisy_loss',mean_loss_noisy,epoch)
    writer.add_scalar('clean_loss',mean_loss_clean,epoch)
    writer.add_scalar('psnr',psnr(outs,cleans),epoch)
    writer.add_image('noisy',make_grid(convert_to_rgb255(noisys)),epoch)
    writer.add_image('clean',make_grid(convert_to_rgb255(cleans)),epoch)
    writer.add_image('out',make_grid(convert_to_rgb255(outs)),epoch)
def main(): 
    for epoch in tqdm(range(5000)):
        total_loss_noisy,total_loss_clean=0,0
        for noisys,cleans in train_loader:
            noisys=noisys.to(device=device)
            cleans=cleans.to(device=device)
            total_loss_noisy+=train(noisys)
            with torch.no_grad():
                total_loss_clean+=restomer.loss(noisys,cleans).sum()
        with torch.no_grad():
            draw(total_loss_noisy,total_loss_clean,noisys,cleans,epoch)
main()